# Guide Notebook

This notebook is designed to provide a guide for the replication project with sample of the data structure, key functionalities of our code, challenges we faced and the result of replication.


In [ ]:
import config
from pathlib import Path
OUTPUT_DIR = Path(config.OUTPUT_DIR)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
x = np.linspace(0, 8 * np.pi, 1000)
y = np.sin(x)
plt.plot(x, y)
filepath = Path(OUTPUT_DIR) / 'sine_graph.png'
plt.savefig(filepath)